In [6]:
import random
import math

In [7]:
class chromosome:
    def __init__(self, number, xs, ys, cal = True) -> None:
        self.n = number + 1
        self.xs = xs
        self.ys = ys
        self.perm = list(range(0, number))
        random.shuffle(self.perm) # generating a (totally!) random permutation of cities.
        self.perm.append(self.perm[0])
        self.fit = 0
        if(cal) :
            self.calc_fit()
    def calc_fit(self): # fittness is sum of distances between each 2 consecutive cities.
        d = 0
        for i in range(self.n - 1) :
            xd = self.xs[self.perm[i + 1]] - self.xs[self.perm[i]]
            yd = self.ys[self.perm[i + 1]] - self.ys[self.perm[i]]
            d += math.sqrt(xd ** 2 + yd ** 2)
        self.fit = d
# mutation : we walk through current permutation and calculate sum of weights of edges co-incident with each vertex and we do a weighted-random choose
# with exponential weights among them. now we have a candid which 'probably' has heavy co-incident edges so we want to swap it with one of neighbors
# of the vertex which has the least distance with it. so we again iterate thruogh points and find the nearest point to our candid and eventually swap
# its position with one of current neighbors of that point. with this approach we are trying to maintain wisdom of algorithm specific to TSP
#  meanwhile not interrupting the role of randomness and probability for choosing which vertices to be swapped.
    def mutation(self, alpha, number):
        if random.random() < alpha:
            # trash 3
            weights = []
            for i in range(len(self.perm)):
                weights.append(0)
            for i in range(1, len(self.perm) - 1):
                xd = self.xs[self.perm[i + 1]] - self.xs[self.perm[i]]
                yd = self.ys[self.perm[i + 1]] - self.ys[self.perm[i]]
                xd2 = self.xs[self.perm[i]] - self.xs[self.perm[i - 1]]
                yd2 = self.ys[self.perm[i]] - self.ys[self.perm[i - 1]]
                weights[i] = (math.sqrt(xd ** 2 + yd ** 2) + math.sqrt(xd2 ** 2 + yd2 ** 2))
            weights[0] = math.sqrt((self.xs[self.perm[0]] - self.xs[self.perm[1]]) ** 2 + (self.ys[self.perm[0]] - self.ys[self.perm[1]]) ** 2)
            weights[self.n - 1] = math.sqrt((self.xs[self.perm[self.n - 1]] - self.xs[self.perm[self.n - 2]]) ** 2 + (self.ys[self.perm[self.n - 1]] - self.ys[self.perm[self.n - 2]]) ** 2)
            candids = random.choices(self.perm[:self.n - 1], k = 1, weights = [math.exp(i // 1000) for i in weights[ : self.n - 1]])
            candid = candids[0]
            ind = self.perm.index(candid)
            ds = []
            for i in range(len(self.xs)):
                if i != candid:
                    xd = self.xs[candid] - self.xs[i]
                    yd = self.ys[candid] - self.ys[i]
                    t = (math.sqrt(xd ** 2 + yd ** 2), i)
                    ds.append(t)
            ds.sort(key = lambda x : x[0])
            ind2 = self.perm.index(ds[0][1])
            if(0 < ind2 and ind2 < number - 1):
                target = ind2 + 1
            elif ind2 == 0:
                target = 1
            elif ind2 == number - 1:
                target = number - 2
            temp = self.perm[target]
            self.perm[target] = self.perm[ind]
            self.perm[ind] = temp
            self.calc_fit()
# cross over : a start and end point which determines the segment of parent2 that goes to child. other segments are supplied by parent1.
# start and end point are chosen randomly.
def cross_over(p1 : chromosome, p2 : chromosome, xs, ys, number) -> chromosome:
    child = chromosome(number, xs, ys, False)
    start = random.randint(0, p1.n - 3)
    end = random.randint(start, p1.n - 2)
    center = p2.perm[start : end + 1]
    left = []
    right = []
    cnt = 0
    for i in range(0, len(p1.perm) - 1):
        if(p1.perm[i] not in center):
            if(cnt < start):
                left.append(p1.perm[i])
            else:
                right.append(p1.perm[i])
            cnt += 1
    child.perm = left + center + right
    child.perm.append(child.perm[0])
    child.calc_fit()
    return child


In [8]:
# In main body of code, we get an iteration number and a population size from the user. Then we start with generating a random primary population.
# (random permutation of cities).then we proceed by making children out of parents in each iteration using cross over and mutation as explianed.
def GA(iterations, pop_size, num_of_cities, xs, ys):
    population = [chromosome(num_of_cities, xs, ys, True) for _ in range(pop_size)]
    population.sort(key = lambda x : x.fit)
    for _ in range (iterations) :
        children = []
        for _ in range(pop_size // 4) :
            parents = random.choices(population, k = 2, weights = [1/(chromosome.fit + 1) for chromosome in population])
            child = cross_over(*parents, xs, ys, num_of_cities)
            child.mutation(1 / (num_of_cities + 1), num_of_cities) # mutation rate : (1 / length of chromosome) as standard genetic algorithm explained in class.
            children.append(child)
        population = population + children
        population.sort(key = lambda x : x.fit) # sort for choosing best individuals
        population = population[ : pop_size]
        print('best_fitness:' , population[0].fit)
    return population[0]

In [9]:
#pr 1002
xs = [1150,  1050,  1150,  1250,  1350,  1050,  3350,  3450,  3550,  3950,  4050,  4050,  4250,  4150,  4450,  4400,  4600,  4900,  5100,  5350,  4950,  4850,  4900,  5000,  5100,  5400,  5750,  5900,  5600,  5400,  5250,  5000,  5000,  5050,  5250,  5450,  5400,  5200,  5050,  4950,  5100,  5200,  5350,  5450,  5600,  5600,  5450,  5350,  5050,  4950,  4700,  4400,  4450,  4100,  4150,  4100,  4300,  4500,  4500,  4700,  4700,  4700,  4600,  4550,  4550,  4300,  4100,  3700,  3550,  3400,  3400,  3700,  3550,  3350,  3350,  3250,  4000,  4100,  3950,  3950,  4200,  4200,  4500,  4700,  4700,  4500,  4500,  4350,  4350,  4247,  4354,  4247,  4100,  4100,  4000,  4199,  4305,  4304,  4500,  4500,  4650,  4800,  5050,  5150,  5300,  5400,  5150,  5250,  5300,  5650,  5800,  5650,  5800,  5750,  5750,  5700,  5700,  5550,  5550,  6000,  6000,  5550,  5400,  5300,  5150,  5150,  5050,  4800,  4650,  4500,  4250,  4250,  4250,  4100,  3950,  3800,  3950,  3800,  3250,  3300,  3300,  3900,  4250,  4400,  4250,  4450,  4650,  4550,  4550,  4650,  4400,  4250,  4550,  4550,  4250,  4300,  4500,  4300,  4300,  4100,  3900,  4200,  4050,  4200,  4050,  3900,  3750,  3450,  3300,  3300,  3450,  3750,  3600,  3750,  3750,  3600,  3900,  3900,  3900,  4100,  4300,  4600,  4600,  4550,  4500,  5600,  5600,  5950,  6100,  6250,  6400,  6200,  6150,  5950,  5650,  5600,  5950,  6050,  6150,  6200,  5950,  5600,  5950,  6050,  6450,  6350,  6550,  6500,  6650,  6950,  6950,  7000,  7200,  7200,  7100,  7050,  6900,  6700,  6850,  7000,  6800,  7000,  7100,  7300,  7300,  7300,  7300,  7000,  6900,  6950,  6700,  6800,  6700,  6700,  6600,  6500,  6400,  6200,  6100,  6100,  5950,  6000,  6150,  6250,  6250,  6350,  6150,  6150,  6000,  6000,  5550,  5550,  5650,  5550,  5550,  5650,  5900,  5950,  6000,  6400,  6450,  6500,  6500,  6600,  6699,  6699,  6805,  6750,  6750,  6600,  6900,  7200,  7350,  7000,  7100,  7100,  6800,  6603,  6498,  6604,  6700,  6600,  6550,  6800,  7100,  7100,  7350,  7350,  7350,  7350,  7350,  7350,  6950,  6850,  6850,  6950,  6650,  6700,  6700,  6600,  6250,  6106,  5999,  5998,  6250,  6100,  6000,  6100,  6100,  6000,  6000,  5850,  5750,  5850,  6000,  6000,  6100,  6148,  6148,  6254,  6450,  6450,  6450,  6254,  6148,  6148,  6450,  6450,  6300,  6150,  6000,  6150,  6300,  6450,  8100,  8200,  8300,  8700,  8800,  8800,  9000,  8900,  9200,  9150,  9350,  9650,  9850,  10100,  9700,  9600,  9650,  9750,  9850,  10150,  10500,  10650,  10350,  10150,  10000,  9750,  9750,  9800,  10000,  10200,  10150,  9950,  9800,  9700,  9850,  9950,  10100,  10200,  10350,  10350,  10200,  10100,  9800,  9700,  9450,  9150,  9200,  8850,  8900,  8850,  9050,  9250,  9250,  9450,  9450,  9450,  9350,  9300,  9300,  9050,  8850,  8450,  8300,  8150,  8150,  8450,  8300,  8100,  8100,  8000,  8750,  8850,  8700,  8700,  8950,  8950,  9250,  9450,  9450,  9250,  9250,  9100,  9100,  8997,  9104,  8997,  8850,  8850,  8750,  8949,  9055,  9054,  9250,  9250,  9400,  9550,  9800,  9900,  10050,  10150,  9900,  10000,  10050,  10400,  10550,  10400,  10550,  10500,  10500,  10450,  10450,  10300,  10300,  10750,  10750,  10300,  10150,  10050,  9900,  9900,  9800,  9550,  9400,  9250,  9000,  9000,  9000,  8850,  8700,  8550,  8700,  8550,  8000,  8050,  8050,  8650,  9000,  9150,  9000,  9200,  9400,  9300,  9300,  9400,  9150,  9000,  9300,  9300,  9000,  9050,  9250,  9050,  9050,  8850,  8650,  8950,  8800,  8950,  8800,  8650,  8500,  8200,  8050,  8050,  8200,  8500,  8350,  8500,  8500,  8350,  8650,  8650,  8650,  8850,  9050,  9350,  9350,  9300,  9250,  10350,  10350,  10700,  10850,  11000,  11150,  10950,  10900,  10700,  10400,  10350,  10700,  10800,  10900,  10950,  10700,  10350,  10700,  10800,  11200,  11100,  11300,  11250,  11400,  11700,  11700,  11750,  11950,  11950,  11850,  11800,  11650,  11450,  11600,  11750,  11550,  11750,  11850,  12050,  12050,  12050,  12050,  11750,  11650,  11700,  11450,  11550,  11450,  11450,  11350,  11250,  11150,  10950,  10850,  10850,  10700,  10750,  10900,  11000,  11000,  11100,  10900,  10900,  10750,  10750,  10300,  10300,  10400,  10300,  10300,  10400,  10650,  10700,  10750,  11150,  11200,  11250,  11250,  11350,  11449,  11449,  11555,  11500,  11500,  11350,  11650,  11950,  12100,  11750,  11850,  11850,  11550,  11353,  11248,  11354,  11450,  11350,  11300,  11550,  11850,  11850,  12100,  12100,  12100,  12100,  12100,  12100,  11700,  11600,  11600,  11700,  11400,  11450,  11450,  11350,  11000,  10856,  10749,  10748,  11000,  10850,  10750,  10850,  10850,  10750,  10750,  10600,  10500,  10600,  10750,  10750,  10850,  10898,  10898,  11004,  11200,  11200,  11200,  11004,  10898,  10898,  11200,  11200,  11050,  10900,  10750,  10900,  11050,  11200,  12850,  12950,  13050,  13450,  13550,  13550,  13750,  13650,  13950,  13900,  14100,  14400,  14600,  14850,  14450,  14350,  14400,  14500,  14600,  14900,  15250,  15400,  15100,  14900,  14750,  14500,  14500,  14550,  14750,  14950,  14900,  14700,  14550,  14450,  14600,  14700,  14850,  14950,  15100,  15100,  14950,  14850,  14550,  14450,  14200,  13900,  13950,  13600,  13650,  13600,  13800,  14000,  14000,  14200,  14200,  14200,  14100,  14050,  14050,  13800,  13600,  13200,  13050,  12900,  12900,  13200,  13050,  12850,  12850,  12750,  13500,  13600,  13450,  13450,  13700,  13700,  14000,  14200,  14200,  14000,  14000,  13850,  13850,  13747,  13854,  13747,  13600,  13600,  13500,  13699,  13805,  13804,  14000,  14000,  14150,  14300,  14550,  14650,  14800,  14900,  14650,  14750,  14800,  15150,  15300,  15150,  15300,  15250,  15250,  15200,  15200,  15050,  15050,  15500,  15500,  15050,  14900,  14800,  14650,  14650,  14550,  14300,  14150,  14000,  13750,  13750,  13750,  13600,  13450,  13300,  13450,  13300,  12750,  12800,  12800,  13400,  13750,  13900,  13750,  13950,  14150,  14050,  14050,  14150,  13900,  13750,  14050,  14050,  13750,  13800,  14000,  13800,  13800,  13600,  13400,  13700,  13550,  13700,  13550,  13400,  13250,  12950,  12800,  12800,  12950,  13250,  13100,  13250,  13250,  13100,  13400,  13400,  13400,  13600,  13800,  14100,  14100,  14050,  14000,  15100,  15100,  15450,  15600,  15750,  15900,  15700,  15650,  15450,  15150,  15100,  15450,  15550,  15650,  15700,  15450,  15100,  15450,  15550,  15950,  15850,  16050,  16000,  16150,  16450,  16450,  16500,  16700,  16700,  16600,  16550,  16400,  16200,  16350,  16500,  16300,  16500,  16600,  16800,  16800,  16800,  16800,  16500,  16400,  16450,  16200,  16300,  16200,  16200,  16100,  16000,  15900,  15700,  15600,  15600,  15450,  15500,  15650,  15750,  15750,  15850,  15650,  15650,  15500,  15500,  15050,  15050,  15150,  15050,  15050,  15150,  15400,  15450,  15500,  15900,  15950,  16000,  16000,  16100,  16199,  16199,  16305,  16250,  16250,  16100,  16400,  16700,  16850,  16500,  16600,  16600,  16300,  16103,  15998,  16104,  16200,  16100,  16050,  16300,  16600,  16600,  16850,  16850,  16850,  16850,  16850,  16850,  16450,  16350,  16350,  16450,  16150,  16200,  16200,  16100,  15750,  15606,  15499,  15498,  15750,  15600,  15500,  15600,  15600,  15500,  15500,  15350,  15250,  15350,  15500,  15500,  15600,  15648,  15648,  15754,  15950,  15950,  15950,  15754,  15648,  15648,  15950,  15950,  15800,  15650,  15500,  15650,  15800,  15950,  6450,  11200,  15950,  5050,  5050,  5050,  9800,  9800,  9800,  14550,  14550,  14550]

In [10]:
#pr 1002
ys = [4000,  2750,  2250,  2050,  2350,  1550,  1700,  1450,  1600,  1700,  2000,  2150,  1650,  1500,  1450,  1700,  1850,  1550,  1550,  1450,  1700,  1900,  2050,  2150,  2050,  2050,  2000,  2050,  2250,  2300,  2250,  2350,  2550,  2800,  2750,  2750,  2950,  3150,  3100,  3300,  3600,  3650,  3750,  3750,  3750,  4250,  4250,  4150,  3800,  3500,  3500,  3700,  3500,  3500,  3300,  3150,  3300,  3150,  2950,  3000,  2800,  2500,  2350,  2500,  2800,  2800,  2950,  2800,  2800,  2700,  3200,  3100,  3300,  4250,  4650,  5200,  5050,  4750,  4650,  4250,  4150,  4550,  4400,  4350,  4750,  4800,  4950,  4750,  4900,  5150,  5256,  5256,  5250,  5350,  5550,  5554,  5554,  5447,  5450,  6050,  6000,  6100,  6050,  6150,  6050,  6050,  5500,  5350,  5150,  5350,  5350,  5500,  5500,  5700,  5850,  6000,  6150,  6000,  6150,  6600,  7000,  6650,  6550,  6450,  6550,  7000,  6550,  6500,  6400,  6550,  6700,  6350,  5950,  5850,  5950,  5950,  6350,  6350,  7000,  7450,  7550,  7850,  7800,  7750,  7300,  7450,  7450,  7650,  7900,  8150,  8250,  8250,  8750,  8950,  9250,  9450,  9750,  9950,  10150,  10150,  10150,  9850,  9700,  9450,  9200,  9250,  9500,  9450,  9350,  10650,  10450,  10500,  11100,  11200,  11400,  11600,  11550,  11400,  11150,  11150,  11150,  11100,  10850,  10550,  10250,  9750,  10250,  10250,  10250,  10200,  10300,  10300,  10400,  10550,  10550,  10850,  10850,  11050,  11050,  11150,  11150,  11100,  11650,  11550,  11550,  11350,  11350,  11150,  11050,  11250,  10650,  10500,  10500,  10200,  10250,  10400,  10450,  10300,  10200,  10050,  9950,  9800,  9750,  9650,  9350,  9150,  9000,  9000,  9000,  9250,  9150,  9350,  9650,  9800,  9800,  10050,  10050,  9950,  10050,  9800,  9750,  9600,  9200,  8950,  8850,  8850,  8600,  8500,  8450,  8600,  8950,  8750,  8150,  7900,  7650,  7450,  7350,  7650,  7850,  8350,  8600,  8700,  9000,  9000,  8856,  8750,  8749,  8600,  8500,  8000,  8350,  8000,  7650,  7350,  6750,  6600,  6000,  6057,  6057,  5950,  5800,  5700,  5500,  5500,  5500,  5100,  4900,  4750,  4150,  4050,  3900,  3800,  3900,  3800,  4050,  4150,  4900,  5050,  5300,  5200,  5500,  5606,  5607,  5500,  5000,  4800,  4450,  4400,  4150,  3950,  3800,  3750,  3450,  3050,  3200,  3300,  3650,  3556,  3450,  3557,  3750,  3650,  3450,  3206,  3206,  3099,  2950,  2800,  2800,  2800,  2800,  2300,  2300,  2300,  1700,  1450,  1600,  1700,  2000,  2150,  1650,  1500,  1450,  1700,  1850,  1550,  1550,  1450,  1700,  1900,  2050,  2150,  2050,  2050,  2000,  2050,  2250,  2300,  2250,  2350,  2550,  2800,  2750,  2750,  2950,  3150,  3100,  3300,  3600,  3650,  3750,  3750,  3750,  4250,  4250,  4150,  3800,  3500,  3500,  3700,  3500,  3500,  3300,  3150,  3300,  3150,  2950,  3000,  2800,  2500,  2350,  2500,  2800,  2800,  2950,  2800,  2800,  2700,  3200,  3100,  3300,  4250,  4650,  5200,  5050,  4750,  4650,  4250,  4150,  4550,  4400,  4350,  4750,  4800,  4950,  4750,  4900,  5150,  5256,  5256,  5250,  5350,  5550,  5554,  5554,  5447,  5450,  6050,  6000,  6100,  6050,  6150,  6050,  6050,  5500,  5350,  5150,  5350,  5350,  5500,  5500,  5700,  5850,  6000,  6150,  6000,  6150,  6600,  7000,  6650,  6550,  6450,  6550,  7000,  6550,  6500,  6400,  6550,  6700,  6350,  5950,  5850,  5950,  5950,  6350,  6350,  7000,  7450,  7550,  7850,  7800,  7750,  7300,  7450,  7450,  7650,  7900,  8150,  8250,  8250,  8750,  8950,  9250,  9450,  9750,  9950,  10150,  10150,  10150,  9850,  9700,  9450,  9200,  9250,  9500,  9450,  9350,  10650,  10450,  10500,  11100,  11200,  11400,  11600,  11550,  11400,  11150,  11150,  11150,  11100,  10850,  10550,  10250,  9750,  10250,  10250,  10250,  10200,  10300,  10300,  10400,  10550,  10550,  10850,  10850,  11050,  11050,  11150,  11150,  11100,  11650,  11550,  11550,  11350,  11350,  11150,  11050,  11250,  10650,  10500,  10500,  10200,  10250,  10400,  10450,  10300,  10200,  10050,  9950,  9800,  9750,  9650,  9350,  9150,  9000,  9000,  9000,  9250,  9150,  9350,  9650,  9800,  9800,  10050,  10050,  9950,  10050,  9800,  9750,  9600,  9200,  8950,  8850,  8850,  8600,  8500,  8450,  8600,  8950,  8750,  8150,  7900,  7650,  7450,  7350,  7650,  7850,  8350,  8600,  8700,  9000,  9000,  8856,  8750,  8749,  8600,  8500,  8000,  8350,  8000,  7650,  7350,  6750,  6600,  6000,  6057,  6057,  5950,  5800,  5700,  5500,  5500,  5500,  5100,  4900,  4750,  4150,  4050,  3900,  3800,  3900,  3800,  4050,  4150,  4900,  5050,  5300,  5200,  5500,  5606,  5607,  5500,  5000,  4800,  4450,  4400,  4150,  3950,  3800,  3750,  3450,  3050,  3200,  3300,  3650,  3556,  3450,  3557,  3750,  3650,  3450,  3206,  3206,  3099,  2950,  2800,  2800,  2800,  2800,  2300,  2300,  2300,  1700,  1450,  1600,  1700,  2000,  2150,  1650,  1500,  1450,  1700,  1850,  1550,  1550,  1450,  1700,  1900,  2050,  2150,  2050,  2050,  2000,  2050,  2250,  2300,  2250,  2350,  2550,  2800,  2750,  2750,  2950,  3150,  3100,  3300,  3600,  3650,  3750,  3750,  3750,  4250,  4250,  4150,  3800,  3500,  3500,  3700,  3500,  3500,  3300,  3150,  3300,  3150,  2950,  3000,  2800,  2500,  2350,  2500,  2800,  2800,  2950,  2800,  2800,  2700,  3200,  3100,  3300,  4250,  4650,  5200,  5050,  4750,  4650,  4250,  4150,  4550,  4400,  4350,  4750,  4800,  4950,  4750,  4900,  5150,  5256,  5256,  5250,  5350,  5550,  5554,  5554,  5447,  5450,  6050,  6000,  6100,  6050,  6150,  6050,  6050,  5500,  5350,  5150,  5350,  5350,  5500,  5500,  5700,  5850,  6000,  6150,  6000,  6150,  6600,  7000,  6650,  6550,  6450,  6550,  7000,  6550,  6500,  6400,  6550,  6700,  6350,  5950,  5850,  5950,  5950,  6350,  6350,  7000,  7450,  7550,  7850,  7800,  7750,  7300,  7450,  7450,  7650,  7900,  8150,  8250,  8250,  8750,  8950,  9250,  9450,  9750,  9950,  10150,  10150,  10150,  9850,  9700,  9450,  9200,  9250,  9500,  9450,  9350,  10650,  10450,  10500,  11100,  11200,  11400,  11600,  11550,  11400,  11150,  11150,  11150,  11100,  10850,  10550,  10250,  9750,  10250,  10250,  10250,  10200,  10300,  10300,  10400,  10550,  10550,  10850,  10850,  11050,  11050,  11150,  11150,  11100,  11650,  11550,  11550,  11350,  11350,  11150,  11050,  11250,  10650,  10500,  10500,  10200,  10250,  10400,  10450,  10300,  10200,  10050,  9950,  9800,  9750,  9650,  9350,  9150,  9000,  9000,  9000,  9250,  9150,  9350,  9650,  9800,  9800,  10050,  10050,  9950,  10050,  9800,  9750,  9600,  9200,  8950,  8850,  8850,  8600,  8500,  8450,  8600,  8950,  8750,  8150,  7900,  7650,  7450,  7350,  7650,  7850,  8350,  8600,  8700,  9000,  9000,  8856,  8750,  8749,  8600,  8500,  8000,  8350,  8000,  7650,  7350,  6750,  6600,  6000,  6057,  6057,  5950,  5800,  5700,  5500,  5500,  5500,  5100,  4900,  4750,  4150,  4050,  3900,  3800,  3900,  3800,  4050,  4150,  4900,  5050,  5300,  5200,  5500,  5606,  5607,  5500,  5000,  4800,  4450,  4400,  4150,  3950,  3800,  3750,  3450,  3050,  3200,  3300,  3650,  3556,  3450,  3557,  3750,  3650,  3450,  3206,  3206,  3099,  2950,  2800,  2800,  2800,  2800,  2300,  2300,  2300,  4950,  4950,  4950,  5750,  8450,  11650,  5750,  8450,  11650,  5750,  8450,  11650]

In [21]:
#pr 1002 result - 500 iterations
res = GA(iterations = 500, pop_size = 100, num_of_cities = 1002, xs = xs, ys = ys)

best_fitness: 6198664.8768866835
best_fitness: 6183149.306548416
best_fitness: 6147406.227346754
best_fitness: 6147406.227346754
best_fitness: 6147406.227346754
best_fitness: 6147406.227346754
best_fitness: 6147406.227346754
best_fitness: 6147406.227346754
best_fitness: 6147406.227346754
best_fitness: 6147406.227346754
best_fitness: 6147406.227346754
best_fitness: 6133202.450969205
best_fitness: 6133202.450969205
best_fitness: 6133202.450969205
best_fitness: 6133202.450969205
best_fitness: 6116935.281252118
best_fitness: 6116935.281252118
best_fitness: 6116935.281252118
best_fitness: 6113694.2220330145
best_fitness: 6113694.2220330145
best_fitness: 6097479.346453511
best_fitness: 6097479.346453511
best_fitness: 6097479.346453511
best_fitness: 6076411.376972604
best_fitness: 6017420.180612177
best_fitness: 6017420.180612177
best_fitness: 6017420.180612177
best_fitness: 6017420.180612177
best_fitness: 6017420.180612177
best_fitness: 6017420.180612177
best_fitness: 6017420.180612177
best_

In [22]:
#pr 1002 result - 500 iterations
res.perm

[922,
 512,
 186,
 374,
 359,
 246,
 475,
 869,
 994,
 163,
 115,
 761,
 910,
 742,
 879,
 542,
 353,
 245,
 369,
 702,
 378,
 202,
 905,
 947,
 1000,
 235,
 456,
 746,
 698,
 783,
 856,
 227,
 838,
 314,
 169,
 182,
 888,
 485,
 26,
 724,
 718,
 620,
 159,
 588,
 31,
 349,
 976,
 895,
 680,
 650,
 765,
 801,
 131,
 984,
 632,
 649,
 963,
 850,
 937,
 909,
 843,
 567,
 553,
 584,
 689,
 959,
 433,
 824,
 925,
 798,
 44,
 547,
 203,
 480,
 208,
 506,
 504,
 647,
 139,
 36,
 404,
 499,
 920,
 696,
 727,
 794,
 354,
 93,
 312,
 157,
 643,
 342,
 323,
 315,
 136,
 368,
 261,
 634,
 813,
 548,
 243,
 248,
 445,
 49,
 893,
 908,
 645,
 59,
 51,
 20,
 436,
 997,
 103,
 251,
 321,
 811,
 459,
 287,
 125,
 171,
 352,
 492,
 0,
 3,
 660,
 621,
 789,
 289,
 64,
 755,
 943,
 964,
 864,
 756,
 30,
 366,
 129,
 14,
 633,
 453,
 519,
 880,
 881,
 700,
 835,
 191,
 252,
 184,
 762,
 790,
 76,
 194,
 836,
 384,
 656,
 333,
 393,
 91,
 635,
 335,
 652,
 195,
 494,
 514,
 301,
 33,
 39,
 18,
 106,
 479,


In [18]:
#pr 1002 result - 300 iterations
res = GA(iterations = 300, pop_size = 100, num_of_cities = 1002, xs = xs, ys = ys)

best_fitness: 6207492.00967934
best_fitness: 6207492.00967934
best_fitness: 6207492.00967934
best_fitness: 6207492.00967934
best_fitness: 6207492.00967934
best_fitness: 6207492.00967934
best_fitness: 6207492.00967934
best_fitness: 6182402.763399133
best_fitness: 6182402.763399133
best_fitness: 6170977.279322478
best_fitness: 6167377.931804527
best_fitness: 6167377.931804527
best_fitness: 6110831.721236015
best_fitness: 6110831.721236015
best_fitness: 6110831.721236015
best_fitness: 6110831.721236015
best_fitness: 6110831.721236015
best_fitness: 6093916.729395056
best_fitness: 6093916.729395056
best_fitness: 6077771.780996528
best_fitness: 6067010.230156471
best_fitness: 6067010.230156471
best_fitness: 6067010.230156471
best_fitness: 6061101.992063536
best_fitness: 6048824.838908514
best_fitness: 6028701.0799113065
best_fitness: 6028701.0799113065
best_fitness: 5948872.443102273
best_fitness: 5948872.443102273
best_fitness: 5948872.443102273
best_fitness: 5948872.443102273
best_fitness:

In [20]:
#pr 1002 result - 300 iterations
res.perm

[806,
 882,
 487,
 224,
 346,
 467,
 200,
 534,
 704,
 618,
 593,
 933,
 721,
 579,
 529,
 783,
 446,
 345,
 495,
 591,
 756,
 565,
 980,
 973,
 909,
 967,
 473,
 334,
 893,
 868,
 175,
 195,
 244,
 814,
 920,
 660,
 360,
 344,
 786,
 989,
 707,
 784,
 162,
 248,
 204,
 652,
 958,
 75,
 89,
 186,
 393,
 280,
 128,
 751,
 516,
 78,
 315,
 87,
 987,
 500,
 819,
 517,
 924,
 914,
 644,
 769,
 840,
 834,
 928,
 629,
 560,
 184,
 368,
 755,
 862,
 795,
 103,
 586,
 424,
 16,
 79,
 277,
 797,
 570,
 878,
 997,
 55,
 314,
 443,
 853,
 877,
 676,
 787,
 455,
 53,
 321,
 107,
 41,
 607,
 444,
 367,
 382,
 698,
 507,
 168,
 203,
 429,
 138,
 83,
 95,
 327,
 809,
 889,
 790,
 35,
 361,
 31,
 792,
 974,
 966,
 836,
 628,
 88,
 441,
 504,
 566,
 85,
 232,
 126,
 52,
 11,
 266,
 766,
 45,
 521,
 569,
 835,
 240,
 349,
 54,
 780,
 602,
 884,
 760,
 539,
 482,
 513,
 729,
 398,
 525,
 979,
 408,
 585,
 955,
 700,
 765,
 864,
 778,
 781,
 81,
 294,
 292,
 466,
 799,
 937,
 956,
 508,
 262,
 236,
 945,


In [1]:
#gr 229
gxs = [68.58, 64.34, 59.55, 59.25, 56.57, 54.43, 54.41, 53.54, 49.5, 50.26, 46.28, 55.45, 56.2, 55.45, 53.12, 51.4, 50, 48.27, 44.36, 47.14, 48.44, 46.21, 41.43, 40.11, 40.23, 58, 56.51, 67.27, 69.2, 55, 55.02, 56.01, 49.5, 43.15, 41.2, 39.4, 38.35, 43.48, 52.16, 47.55, 52.03, 62.13, 64.45, 53.01, 59.34, 50.17, 50.35, 48.27, 46.58, 43.1, 41.01, 38.25, 39.56, 38.43, 39.45, 39.55, 37.55, 37.01, 36.12, 34.44, 33.3, 33.53, 31.57, 32.5, 32.04, 31.46, 24.28, 21.3, 21.27, 15.23, 14.48, 12.45, 14.32, 23.37, 25.18, 25.17, 26.13, 24.38, 29.2, 30.3, 33.21, 35.28, 36.2, 38.05, 37.16, 35.4, 34.19, 30.2, 32.4, 29.36, 30.17, 36.18, 34.2, 31.32, 34.31, 33.36, 31.35, 31.25, 30.11, 30.12, 27.42, 25.22, 24.52, 30.19, 28.4, 26.17, 26.55, 26.28, 25.2, 25.36, 22.32, 23.02, 21.09, 20.3, 18.58, 17.23, 17.42, 15.21, 12.59, 13.05, 10.49, 9.56, 6.56, 27.43, 27.28, 23.43, 22.2, 22, 16.47, 18.47, 19.52, 17.58, 21.02, 16.28, 16.04, 10.45, 11.33, 13.45, 5.25, 3.1, 1.17, 3.35, -0.57, -2.55, -6.1, -6.54, -7.48, -7.15, -8.39, -10.1, -3.2, 1.33, 4.56, -0.3, -5.07, 1.29, -3.43, -5.4, 7.04, 10.18, 10.42, 14.35, 22.17, 22.38, 25.03, 29.4, 36.03, 34.15, 30.39, 29.39, 25.05, 23.06, 26.06, 30.36, 32.03, 31.14, 34.48, 36.06, 37.55, 39.08, 39.55, 38.53, 41.48, 45.45, 39.01, 37.33, 35.06, 43.03, 39.43, 38.15, 35.42, 35.1, 36.34, 35, 34.4, 34.24, 32.48, 31.36, 26.13, 13.28, -2.32, -4.12, -9.3, -12.28, -31.56, -34.55, -37.49, -42.53, -33.52, -27.28, -19.16, -23.42, -45.52, -43.32, -41.18, -36.52, -21.08, -14.16, -18.08, -22.16, -9.26, -0.32, 11.35, 21.19, 1.52, -9.45, -17.32, -25.04, -27.07]

In [2]:
#gr 229
gys = [33.05, 40.32, 30.15, 24.45, 24.06, 20.3, 25.19, 27.34, 24, 30.31, 30.44, 37.35, 44, 49.08, 50.09, 39.1, 36.15, 34.59, 33.32, 39.42, 44.25, 48.03, 44.49, 44.3, 49.51, 56.15, 60.36, 63.58, 88.06, 73.24, 82.55, 92.5, 73.1, 76.57, 69.18, 66.48, 68.48, 87.35, 104.2, 106.53, 113.3, 129.49, 177.29, 158.39, 150.48, 127.32, 137.02, 135.06, 142.42, 131.56, 28.58, 27.09, 32.52, 35.3, 37.02, 41.17, 40.14, 35.18, 37.1, 36.43, 36.18, 35.3, 35.56, 35, 34.46, 35.14, 39.36, 39.12, 39.49, 44.12, 42.57, 45.12, 49.08, 58.35, 55.18, 51.32, 50.35, 46.43, 47.59, 47.47, 44.25, 44.28, 43.08, 46.18, 49.36, 51.26, 47.04, 48.16, 51.38, 52.32, 57.05, 59.36, 62.12, 65.3, 69.12, 73.04, 74.18, 73.05, 71.29, 67, 68.52, 68.22, 67.03, 78.02, 77.13, 73.02, 75.49, 80.21, 83, 85.07, 88.22, 72.37, 79.06, 85.5, 72.5, 78.29, 83.18, 75.1, 77.35, 80.17, 78.41, 78.07, 79.51, 85.19, 89.39, 90.25, 91.5, 96.05, 96.1, 98.59, 102.08, 102.36, 105.51, 107.36, 108.13, 106.4, 104.55, 100.31, 100.2, 101.42, 103.51, 98.4, 100.21, 104.45, 106.48, 107.36, 110.22, 112.45, 115.13, 123.35, 114.35, 110.2, 114.55, 117.09, 119.24, 124.51, 128.12, 132.45, 125.36, 123.54, 122.34, 121, 114.09, 120.17, 121.3, 91.09, 103.41, 108.52, 104.04, 106.34, 102.4, 113.16, 119.17, 114.17, 118.47, 121.28, 113.39, 120.19, 112.3, 117.12, 116.25, 121.35, 123.27, 126.41, 125.45, 126.58, 129.03, 141.21, 140.07, 140.53, 139.46, 136.55, 136.39, 135.45, 135.3, 132.27, 129.55, 130.33, 127.4, 144.47, 140.42, 152.12, 147.1, 130.5, 115.5, 138.35, 144.58, 147.19, 151.13, 153.02, 146.48, 133.53, 170.3, 172.38, 174.47, 174.46, -175.12, -170.42, 178.25, 166.27, 159.57, 166.55, 165.23, -157.52, -157.2, -139, -149.34, -130.06, -109.22]

In [12]:
#gr 229 result - 500 iterations
res = GA(iterations = 500, pop_size = 200, num_of_cities = 229, xs = gxs, ys = gys)

best_fitness: 13758.4897197813
best_fitness: 13758.4897197813
best_fitness: 13758.4897197813
best_fitness: 13758.4897197813
best_fitness: 13758.4897197813
best_fitness: 13758.4897197813
best_fitness: 13758.4897197813
best_fitness: 13758.4897197813
best_fitness: 13758.4897197813
best_fitness: 13699.459516802588
best_fitness: 13699.459516802588
best_fitness: 13699.459516802588
best_fitness: 13465.200840379508
best_fitness: 13465.200840379508
best_fitness: 13465.200840379508
best_fitness: 13465.200840379508
best_fitness: 13455.318763315905
best_fitness: 13455.318763315905
best_fitness: 13455.318763315905
best_fitness: 13446.289865554025
best_fitness: 13446.289865554025
best_fitness: 13446.289865554025
best_fitness: 13294.980954365144
best_fitness: 13294.980954365144
best_fitness: 13294.980954365144
best_fitness: 13294.980954365144
best_fitness: 13260.003437493522
best_fitness: 13260.003437493522
best_fitness: 13260.003437493522
best_fitness: 13260.003437493522
best_fitness: 12961.55485825

In [14]:
#gr 229 result - 500 iterations
res.perm

[218,
 219,
 203,
 227,
 225,
 224,
 216,
 116,
 157,
 209,
 214,
 191,
 129,
 32,
 153,
 121,
 119,
 188,
 177,
 223,
 228,
 226,
 217,
 16,
 73,
 23,
 155,
 143,
 120,
 130,
 25,
 19,
 81,
 80,
 7,
 6,
 4,
 104,
 37,
 174,
 159,
 149,
 136,
 45,
 176,
 180,
 46,
 148,
 97,
 88,
 27,
 107,
 141,
 206,
 207,
 208,
 201,
 42,
 43,
 34,
 125,
 131,
 183,
 186,
 185,
 184,
 197,
 40,
 168,
 126,
 108,
 117,
 154,
 156,
 151,
 142,
 124,
 150,
 162,
 138,
 49,
 164,
 132,
 13,
 20,
 0,
 22,
 35,
 85,
 90,
 86,
 96,
 33,
 163,
 190,
 195,
 194,
 114,
 94,
 55,
 66,
 21,
 1,
 15,
 62,
 12,
 5,
 87,
 83,
 68,
 175,
 147,
 205,
 101,
 79,
 115,
 165,
 215,
 212,
 158,
 193,
 189,
 145,
 200,
 98,
 118,
 70,
 56,
 53,
 91,
 10,
 31,
 38,
 196,
 178,
 172,
 48,
 30,
 65,
 52,
 76,
 69,
 67,
 134,
 169,
 139,
 51,
 18,
 2,
 17,
 110,
 137,
 179,
 39,
 182,
 44,
 192,
 41,
 199,
 47,
 181,
 161,
 202,
 222,
 160,
 144,
 113,
 103,
 84,
 60,
 8,
 9,
 50,
 54,
 102,
 95,
 28,
 93,
 14,
 82,
 77,
 57

In [15]:
#gr 229 result - 1000 iterations
res = GA(iterations = 1000, pop_size = 100, num_of_cities = 229, xs = gxs, ys = gys)

best_fitness: 14535.704396666124
best_fitness: 14535.704396666124
best_fitness: 14535.704396666124
best_fitness: 14326.749079239258
best_fitness: 14326.749079239258
best_fitness: 14326.749079239258
best_fitness: 14326.749079239258
best_fitness: 13716.981828461387
best_fitness: 13716.981828461387
best_fitness: 13716.981828461387
best_fitness: 13716.981828461387
best_fitness: 13716.981828461387
best_fitness: 13716.981828461387
best_fitness: 13716.981828461387
best_fitness: 13716.981828461387
best_fitness: 13381.722030753783
best_fitness: 13381.722030753783
best_fitness: 13381.722030753783
best_fitness: 13381.722030753783
best_fitness: 13381.722030753783
best_fitness: 13381.722030753783
best_fitness: 13381.722030753783
best_fitness: 13319.40978150244
best_fitness: 13129.675961453539
best_fitness: 13129.675961453539
best_fitness: 13129.675961453539
best_fitness: 13129.675961453539
best_fitness: 13129.675961453539
best_fitness: 13129.675961453539
best_fitness: 13129.675961453539
best_fitnes

In [16]:
#gr 229 result - 1000 iterations
res.perm

[57,
 63,
 177,
 186,
 203,
 207,
 215,
 216,
 217,
 225,
 53,
 24,
 19,
 48,
 97,
 117,
 114,
 198,
 185,
 29,
 2,
 108,
 121,
 76,
 74,
 32,
 188,
 164,
 227,
 224,
 71,
 87,
 134,
 202,
 211,
 156,
 38,
 11,
 226,
 223,
 228,
 4,
 22,
 81,
 58,
 80,
 3,
 52,
 50,
 65,
 84,
 70,
 99,
 104,
 152,
 163,
 199,
 169,
 129,
 142,
 144,
 147,
 151,
 102,
 62,
 17,
 16,
 10,
 96,
 90,
 85,
 56,
 26,
 39,
 167,
 138,
 150,
 77,
 112,
 119,
 136,
 201,
 218,
 125,
 137,
 184,
 182,
 162,
 213,
 214,
 158,
 171,
 95,
 36,
 92,
 100,
 111,
 165,
 168,
 176,
 178,
 42,
 43,
 44,
 193,
 157,
 133,
 130,
 105,
 101,
 69,
 82,
 33,
 166,
 27,
 89,
 20,
 59,
 61,
 75,
 21,
 23,
 9,
 1,
 0,
 31,
 183,
 190,
 173,
 107,
 68,
 79,
 15,
 14,
 13,
 30,
 180,
 159,
 143,
 139,
 145,
 155,
 174,
 45,
 191,
 194,
 195,
 206,
 219,
 221,
 220,
 222,
 175,
 109,
 110,
 123,
 37,
 98,
 54,
 18,
 5,
 55,
 34,
 40,
 179,
 122,
 120,
 116,
 103,
 66,
 72,
 106,
 140,
 153,
 205,
 212,
 210,
 204,
 118,
 113,
 126